In [100]:
import simpy
import random
import pandas as pd

In [101]:
class Cliente:

    def __init__( self, env, id, cajero ):
        self.env = env
        self.id = id
        self.cajero = cajero

        self.df = pd.DataFrame()

    def start(self):
        print( f" {self.id} llega al banco en tiempo: { self.env.now } " )
        t = pd.DataFrame({
            "nombre": [self.id]
            , "tiempo": [self.env.now]
            , "evento": ["llega"]
        })

        with self.cajero.request() as req:
            yield req
            print( f" {self.id} comienza transacciones en tiempo: { self.env.now } " )
            t2 = pd.DataFrame({
                "nombre": [self.id]
                , "tiempo": [self.env.now]
                , "evento": ["inicia"]
            })

            yield self.env.timeout( random.randint( 5, 11 ) )
            print( f" {self.id} termina transacciones en tiempo: { self.env.now } " )
            t3 = pd.DataFrame({
                "nombre": [self.id]
                , "tiempo": [self.env.now]
                , "evento": ["fin"]
            })

            self.df = pd.concat([t, t2, t3], ignore_index=True)

In [102]:
def cliente( env, nombre, cajero ):
    print( f" {nombre} llega al banco en tiempo: { env.now } " )
    t = pd.DataFrame({
        "nombre": [nombre]
        , "tiempo": [env.now]
        , "evento": ["llega"]
    })

    with cajero.request() as req:
        yield req
        print( f" {nombre} comienza transacciones en tiempo: { env.now } " )
        t2 = pd.DataFrame({
            "nombre": [nombre]
            , "tiempo": [env.now]
            , "evento": ["inicia"]
        })

        yield env.timeout( random.randint( 5, 11 ) )
        print( f" {nombre} termina transacciones en tiempo: { env.now } " )
        t3 = pd.DataFrame({
            "nombre": [nombre]
            , "tiempo": [env.now]
            , "evento": ["fin"]
        })

        return pd.concat( [ t, t2, t3], ignore_index=True )

In [103]:
def generate_clientes(env, n, cajero):
    for i in range(n):
        yield env.timeout( random.randint( 2, 4 ) )
        c1 = Cliente( env, f"Cliente {i}", cajero )
        env.process( c1.start() )
        print( c1.df )

In [104]:
class Generar:
    def __init__( self, env, n_clientes, n_cajeros ):

        self.env = env
        self.cajero = simpy.Resource( env, capacity=n_cajeros )
        self.n_clientes = n_clientes
        self.clientes = []

        self.env.process( self.start() )



    def start(self):
        for i in range(self.n_clientes):
            yield self.env.timeout( random.randint( 2, 4 ) )
            c1 = Cliente( self.env, f"Cliente {i}", self.cajero )
            self.env.process( c1.start() )
            self.clientes.append( c1 )




In [105]:
env = simpy.Environment()
gc = Generar( env, 5, 2 )
env.run( until=200 )

 Cliente 0 llega al banco en tiempo: 2 
 Cliente 0 comienza transacciones en tiempo: 2 
 Cliente 1 llega al banco en tiempo: 5 
 Cliente 1 comienza transacciones en tiempo: 5 
 Cliente 2 llega al banco en tiempo: 7 
 Cliente 0 termina transacciones en tiempo: 10 
 Cliente 3 llega al banco en tiempo: 10 
 Cliente 2 comienza transacciones en tiempo: 10 
 Cliente 4 llega al banco en tiempo: 12 
 Cliente 1 termina transacciones en tiempo: 16 
 Cliente 3 comienza transacciones en tiempo: 16 
 Cliente 2 termina transacciones en tiempo: 21 
 Cliente 3 termina transacciones en tiempo: 21 
 Cliente 4 comienza transacciones en tiempo: 21 
 Cliente 4 termina transacciones en tiempo: 32 


In [106]:
eda = pd.concat( [ c.df for c in gc.clientes ], ignore_index=True )

In [107]:
inicio = eda.loc[ eda["evento"] == "inicia" , ["nombre", "tiempo"]  ]
llega = eda.loc[ eda["evento"] == "llega" , ["nombre", "tiempo"]  ]

In [108]:
p1 = inicio.merge(
    llega
    , on=["nombre"]
    , how="inner"
)

In [109]:
p1["espera"] = p1["tiempo_x"] - p1["tiempo_y"]

In [111]:
p1["espera"].mean()

np.float64(3.6)